In [7]:
# Analyze the log for testing
def analyze_log_data(filename):
        """
        Analyzes log data related to websocket sessions and user activity.

        Args:
          filename: Path to the file containing the log messages.

        Prints insights about the user's session activity and possible reasons for termination.
        """
        # Initialize counters
        session_starts = 0
        session_ends = 0
        unsubscribes = 0
        data_requests = 0

        # Open the log file
        with open(filename, 'r') as f:
            for line in f:
                # Parse the log message
                # This part might require adjustments based on your specific log format
                if "terminate" in line:
                    session_starts += 1
                elif "requests market data" in line:
                    data_requests += 1
                elif "unsubscribes from the session" in line:
                    unsubscribes += 1
                elif "session is terminated" in line:
                    session_ends += 1

        # Print insights
        print(f"Total session starts: {session_starts}")
        print(f"Total data requests: {data_requests}")
        print(f"Total unsubscribes: {unsubscribes}")
        print(f"Total session terminations: {session_ends}")

        # Check for potential reasons behind terminations
        if session_ends > session_starts:
            print("Possible reason for termination: User inactivity or network issues.")
        if unsubscribes > 0:
            print("Possible reason for termination: Application logic requiring session refresh.")

        # Additional analysis (optional)
        # You can extend this script to perform more complex analysis, like:
        # - Calculate average session duration
        # - Analyze timestamps to identify inactivity periods
        # - Correlate terminations with specific events
# Replace 'path/to/your/log/file.txt' with the actual file path

analyze_log_data('/Log_Investigation/tcfeedservice-solactive/tcfeedservice-solactive/combinded_log.log')

Total session starts: 239
Total data requests: 0
Total unsubscribes: 0
Total session terminations: 0


In [9]:
def analyze_log(filename):
    # Initialize counters

    session_start = 0
    session_end = 0
    session_unsub = 0

    #Open the log file
    with open(filename,'r') as file:
        for line in file:
            if "session started" in line :
                session_start +=1
            elif "terminate WsKey(UserCode(solactive))" in line :
                session_end +=1
            elif "session actor terminated for UserCode(solactive) - removing session from data structures" in line :
                session_unsub +=1

    # print insight check
        print(f"Total session starts: {session_start}")
        print(f"Total terminate WsKey(UserCode(solactive)): {session_end}")
        print(f"Total unsubscribes: {session_unsub}")
        if session_end > session_start:
             print("Possible reason for termination: User inactivity or network issues.")
        if session_unsub > 0 :
            print("Possible reason for termination: Application logic requiring session refresh")

analyze_log('/Log_Investigation/tcfeedservice-solactive/tcfeedservice-solactive/combinded_log.log')



Total session starts: 80
Total terminate WsKey(UserCode(solactive)): 159
Total unsubscribes: 80
Possible reason for termination: User inactivity or network issues.
Possible reason for termination: Application logic requiring session refresh


In [11]:
def analyze_log_data(filename):

    # Initialize counters and variables
    session_starts = {}  # Dictionary to store session start times (timestamp)
    session_ends = {}    # Dictionary to store session end times (timestamp)
    data_requests = 0
    unsubscribes = 0

    # Open the log file
    with open(filename, 'r') as f:
        for line in f:
            # Parse the log message
            timestamp, message = parse_log_message(line)  # Implement parse_log_message function

            # Update counters based on message type
            if "starts a new session" in message:
                session_starts[timestamp] = timestamp  # Start time is the timestamp itself
            elif "requests market data" in line:
                data_requests += 1
            elif "unsubscribes from the session" in line:
                unsubscribes += 1
            elif "session is terminated" in line:
                session_ends[timestamp] = timestamp  # End time is the timestamp itself

    # Calculate session durations (assuming timestamps are in seconds)
    total_duration = 0
    session_count = len(session_starts)
    for start_time, end_time in zip(session_starts.values(), session_ends.values()):
        if end_time:  # Check if session has an end time (might not for all starts)
            duration = end_time - start_time
            total_duration += duration

    # Calculate average session duration (if there were any sessions)
    average_duration = total_duration / session_count if session_count > 0 else 0

    # Identify inactivity periods (assuming timestamps are in seconds)
    inactivity_threshold = 60  # Set inactivity threshold in seconds (e.g., 1 minute)
    inactivity_periods = []
    previous_timestamp = None
    for timestamp in sorted(session_starts.keys() | session_ends.keys()):
        if previous_timestamp and (timestamp - previous_timestamp) > inactivity_threshold:
            inactivity_periods.append((previous_timestamp, timestamp))
        previous_timestamp = timestamp

    # Print insights
    print(f"Total session starts: {session_count}")
    print(f"Total data requests: {data_requests}")
    print(f"Total unsubscribes: {unsubscribes}")
    print(f"Total session terminations: {len(session_ends)}")
    print(f"Average session duration: {average_duration:.2f} seconds")

    if inactivity_periods:
        print(f"Inactivity periods (above {inactivity_threshold} seconds):")
        for start, end in inactivity_periods:
            print(f"- {start} to {end}")
    else:
        print("No inactivity periods detected.")

    # Potential reasons for termination (same as before)
    # ... (refer to previous code)

# Implement parse_log_message function to extract timestamp and message from each log line
def parse_log_message(line):
    # Extract timestamp and message based on your specific log format
    # This function is left as an exercise for you
    # You can use regular expressions or string manipulation techniques
    # ...
    pass

analyze_log_data('/Log_Investigation/tcfeedservice-solactive/tcfeedservice-solactive/combinded_log.log')


TypeError: argument of type 'NoneType' is not iterable

In [ ]:
def parse_log_message(line):
    """
    Extracts timestamp and message from a log line.
    Args:
      line: A single line from the log file.
    Returns:
      A tuple containing the timestamp (as integer) and message (as string).
    """
    # Split the line based on spaces (assuming timestamp is separated by spaces)
    parts = line.split()

    # Check if the line has enough parts (at least 2 for timestamp and message)
    if len(parts) < 2:
        return None, None

    # Extract timestamp (assuming first part is a timestamp)
    try:
        timestamp = int(parts[0])
    except ValueError:
        # Handle cases where the first part isn't an integer (e.g., unexpected log format)
        return None, None

    # Reconstruct the message part (assuming everything after timestamp is the message)
    message = " ".join(parts[1:])

    return timestamp, message